Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [5]:
import pandas as pd
import numpy as np

Przygotowanie macierzy interakcji

In [6]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)

df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1
heatMapDF = pd.pivot_table(df, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
heatMapDF

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,2,4,16,1,3,2,5,3,9,...,0,1,2,1,1,3,6,1,0,7
104,0,1,6,6,6,9,0,5,3,6,...,1,2,1,0,1,1,7,2,6,1
105,0,2,7,10,4,5,0,4,3,5,...,0,1,2,0,2,1,10,5,0,1
106,3,1,2,9,2,3,2,3,1,6,...,0,0,2,3,1,1,6,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,3,1,6,14,8,2,0,3,8,6,...,0,2,0,2,0,1,10,1,3,3
298,0,0,8,1,3,3,0,6,1,2,...,1,0,0,0,1,2,9,3,0,0
299,0,0,3,0,1,0,0,1,0,1,...,0,0,1,0,2,1,3,1,0,1
